In [22]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


sns.set()

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
# from IPython.core.display import display, HTML
from IPython.display import display, HTML

display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

import warnings
warnings.filterwarnings(action='ignore')

In [23]:
# import pandas_profiling

# ----------------- 학습 ----------------------
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# ----------------- 모델 ----------------------
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge

# ----------------- 평가 ----------------------
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score
# from sklearn.metrics import precision_recall_curve, plot_precision_recall_curve

# from sklearn.metrics import roc_auc_score, roc_curve, plot_roc_curve

# 데이터 로드

In [24]:
train = pd.read_csv('/content/drive/MyDrive/Chat NBT(파이널)/data/train_test/traindf.csv')
test = pd.read_csv('/content/drive/MyDrive/Chat NBT(파이널)/data/train_test/testdf.csv')
train = train.rename(columns={'Unnamed: 0':'날짜'})
test = test.rename(columns={'Unnamed: 0':'날짜'})

# train.info(),test.info()

In [25]:
df = pd.concat([train,test], axis=0)

# H2o

In [26]:
# pip install h2o

In [27]:
import h2o

In [28]:
from h2o.automl import H2OAutoML

In [29]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 min 34 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,1 month and 22 days
H2O_cluster_name:,H2O_from_python_unknownUser_vwksdy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [38]:
x= list(df[['장중_긍정비율','장중_부정비율','장중_긍부정비율','장외_긍정비율','장외_부정비율','장외_긍부정비율']])
y ='등락여부'
# X_test = test[['장중_긍정비율','장중_부정비율','장중_긍부정비율','장외_긍정비율','장외_부정비율','장외_긍부정비율']]
# y_test= test['등락률']

In [39]:
h2o_train = h2o.H2OFrame(train)
h2o_test= h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [41]:
# For binary classification, response should be a factor
h2o_train[y] = h2o_train[y].asfactor()
h2o_test[y] = h2o_test[y].asfactor()

In [42]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=h2o_train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_3_AutoML_1_20230620_12110_model_1


Status of Neuron Layers: predicting 등락여부, 2-class classification, bernoulli distribution, CrossEntropy loss, 21,102 weights/biases, 255.7 KB, 548,340 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms           mean_bias              bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  -------------------  ---------------------  -------------------
    1        6        Input             15.0
    2        100      RectifierDropout  10.0       0.0   0.0   0.0019361448340350761  0.0016834582202136517  0.0         0.002205699110830513  0.2456507682800293   0.12470132755011627    0.29315829277038574
    3        100      RectifierDropout  10.0       0.0   0.0   0.012460787282319507   0.01569914072751999    0.0         -0.0741950918524768   0.15066003799438477  1.0118664065922411     0.3038138151168823
    4        100      RectifierDropout  10.0       0.0   0.0   0.009150037860957673   0.02743683010339737    0.0         -0.05006441508950247  0.10680606961250305  0.7351303881175191     0.20990514755249023
    5        2        Softmax                      0.0   0.0   0.002746259350096807   0.0021316073834896088  0.0         -0.02890964059155522  0.5416815280914307   0.0013715291492738813  0.2232486605644226

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.1572081958171847
RMSE: 0.396494887504473
LogLoss: 0.4836320946821173
Mean Per-Class Error: 0.22457710464201416
AUC: 0.8540273406766327
AUCPR: 0.8796019758428338
Gini: 0.7080546813532653

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.24054891701801365
       0    1    Error    Rate
-----  ---  ---  -------  -------------
0      185  63   0.254    (63.0/248.0)
1      48   198  0.1951   (48.0/246.0)
Total  233  261  0.2247   (111.0/494.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.240549     0.781065  192
max f2                       0.0695407    0.847339  356
max f0point5                 0.567968     0.820399  103
max accuracy                 0.384469     0.791498  151
max precision                0.999991     1         0
max recall                   0.0335806    1         383
max specificity              0.999991     1         0
max absolute_mcc             0.384469     0.586321  151
max min_per_class_accuracy   0.279699     0.770161  178
max mean_per_class_accuracy  0.384469     0.791273  151
max tns                      0.999991     248       0
max fns                      0.999991     230       0
max fps                      0.000888513  248       399
max tps                      0.0335806    246       383
max tnr                      0.999991     1         0
max fnr                      0.999991     0.934959  0
max fpr                      0.000888513  1         399
max tpr                      0.0335806    1         383

Gains/Lift Table: Avg response rate: 49.80 %, avg score: 44.61 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0101215                   0.999999           2.00813   2.00813            1                1          1                      

In [43]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
DeepLearning_grid_3_AutoML_1_20230620_12110_model_1,0.820245,0.548803,0.837519,0.262752,0.422021,0.178102
DeepLearning_grid_2_AutoML_1_20230620_12110_model_1,0.818647,0.549597,0.843252,0.270833,0.423239,0.179131
StackedEnsemble_BestOfFamily_1_AutoML_1_20230620_12110,0.817753,0.517334,0.834426,0.27085,0.41263,0.170263
GLM_1_AutoML_1_20230620_12110,0.815909,0.534356,0.843174,0.230904,0.419785,0.17622
StackedEnsemble_AllModels_1_AutoML_1_20230620_12110,0.811467,0.517705,0.832973,0.272849,0.413076,0.170632
DeepLearning_grid_1_AutoML_1_20230620_12110_model_1,0.807673,0.592122,0.839602,0.246935,0.4253,0.18088
DeepLearning_1_AutoML_1_20230620_12110,0.804075,0.599857,0.828942,0.230904,0.431536,0.186223
GBM_grid_1_AutoML_1_20230620_12110_model_1,0.80092,0.533811,0.822451,0.266949,0.422157,0.178216
GBM_grid_1_AutoML_1_20230620_12110_model_2,0.800747,0.526266,0.829681,0.241018,0.419181,0.175712
GBM_2_AutoML_1_20230620_12110,0.800395,0.53287,0.826846,0.260982,0.422628,0.178615


<pre>
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.24054891701801365
0	1	Error	Rate
0	185.0	63.0	0.254	(63.0/248.0)
1	48.0	198.0	0.1951	(48.0/246.0)
Total	233.0	261.0	0.2247	(111.0/494.0)


In [44]:
aml.leader
# RMSE: 0.9239857315864636

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_3_AutoML_1_20230620_12110_model_1


Status of Neuron Layers: predicting 등락여부, 2-class classification, bernoulli distribution, CrossEntropy loss, 21,102 weights/biases, 255.7 KB, 548,340 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms           mean_bias              bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  -------------------  ---------------------  -------------------
    1        6        Input             15.0
    2        100      RectifierDropout  10.0       0.0   0.0   0.0019361448340350761  0.0016834582202136517  0.0         0.002205699110830513  0.2456507682800293   0.12470132755011627    0.29315829277038574
    3        100      RectifierDropout  10.0       0.0   0.0   0.012460787282319507   0.01569914072751999    0.0         -0.0741950918524768   0.15066003799438477  1.0118664065922411     0.3038138151168823
    4        100      RectifierDropout  10.0       0.0   0.0   0.009150037860957673   0.02743683010339737    0.0         -0.05006441508950247  0.10680606961250305  0.7351303881175191     0.20990514755249023
    5        2        Softmax                      0.0   0.0   0.002746259350096807   0.0021316073834896088  0.0         -0.02890964059155522  0.5416815280914307   0.0013715291492738813  0.2232486605644226

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.1572081958171847
RMSE: 0.396494887504473
LogLoss: 0.4836320946821173
Mean Per-Class Error: 0.22457710464201416
AUC: 0.8540273406766327
AUCPR: 0.8796019758428338
Gini: 0.7080546813532653

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.24054891701801365
       0    1    Error    Rate
-----  ---  ---  -------  -------------
0      185  63   0.254    (63.0/248.0)
1      48   198  0.1951   (48.0/246.0)
Total  233  261  0.2247   (111.0/494.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.240549     0.781065  192
max f2                       0.0695407    0.847339  356
max f0point5                 0.567968     0.820399  103
max accuracy                 0.384469     0.791498  151
max precision                0.999991     1         0
max recall                   0.0335806    1         383
max specificity              0.999991     1         0
max absolute_mcc             0.384469     0.586321  151
max min_per_class_accuracy   0.279699     0.770161  178
max mean_per_class_accuracy  0.384469     0.791273  151
max tns                      0.999991     248       0
max fns                      0.999991     230       0
max fps                      0.000888513  248       399
max tps                      0.0335806    246       383
max tnr                      0.999991     1         0
max fnr                      0.999991     0.934959  0
max fpr                      0.000888513  1         399
max tpr                      0.0335806    1         383

Gains/Lift Table: Avg response rate: 49.80 %, avg score: 44.61 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0101215                   0.999999           2.00813   2.00813            1                1          1                      